In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer


In [5]:
base_model = "NousResearch/Llama-2-7b-chat-hf"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [8]:
from peft import LoraConfig, TaskType, get_peft_model
lora_config = LoraConfig(r=8, target_modules = ["q_proj", "o_proj","k_proj", "v_proj", "gate_proj", "up_proj", "down_proj" ],
                         bias = "none", task_type = TaskType.CAUSAL_LM,)

In [9]:
model = get_peft_model(model, lora_config)

In [10]:
def tokenize_function(examples):
    inputs = examples['Essay']
    targets = examples['Feedback']
    inputs = [tokenizer.bos_token + inp + tokenizer.eos_token for inp in inputs]
    targets = [tokenizer.bos_token + tgt + tokenizer.eos_token for tgt in targets]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [11]:
dataset = load_dataset("Atharva1244/college_essays")

Generating train split:   0%|          | 0/85 [00:00<?, ? examples/s]

In [12]:
dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['Essay_cleaned', 'Feedback_cleaned'])
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [14]:
training_args = TrainingArguments(
    output_dir='./essayfeedback',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
from transformers import Trainer, default_data_collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test'],
    tokenizer=tokenizer,
    data_collator=default_data_collator)

In [16]:
trainer.train()

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Epoch,Training Loss,Validation Loss
1,13.758900,12.871562
2,9.295000,9.831588


TrainOutput(global_step=68, training_loss=13.085046655991498, metrics={'train_runtime': 134.7065, 'train_samples_per_second': 1.01, 'train_steps_per_second': 0.505, 'total_flos': 2768846331052032.0, 'train_loss': 13.085046655991498, 'epoch': 2.0})

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,8.300200,6.371289
2,5.016600,5.659576


TrainOutput(global_step=68, training_loss=6.886055385365205, metrics={'train_runtime': 133.4332, 'train_samples_per_second': 1.019, 'train_steps_per_second': 0.51, 'total_flos': 2768846331052032.0, 'train_loss': 6.886055385365205, 'epoch': 2.0})

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.792400,3.763183
2,3.057100,3.351079


TrainOutput(global_step=68, training_loss=4.031521292293773, metrics={'train_runtime': 133.4355, 'train_samples_per_second': 1.019, 'train_steps_per_second': 0.51, 'total_flos': 2768846331052032.0, 'train_loss': 4.031521292293773, 'epoch': 2.0})

In [22]:
trainer.push_to_hub()


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/80.0M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Atharva1244/essayfeedback/commit/3229b969b794f379768f9bf1a7a8868c6d56aff5', commit_message='End of training', commit_description='', oid='3229b969b794f379768f9bf1a7a8868c6d56aff5', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
from huggingface_hub import notebook_login
notebook_login()